In [2]:
## Iniciar BeamNG
import time
import keyboard
from pathlib import Path
from datetime import datetime
from beamngpy import BeamNGpy, Scenario, Vehicle
from beamngpy.sensors import Electrics

def millis():
    return time.time_ns() / 1000000
oldtime = millis()
curtime = 0
rectime = 0
step = 1/60 * 1000 #60 hz

bng_path = "C:\Program Files (x86)\Steam\steamapps\common\BeamNG.drive"
bng = BeamNGpy('localhost', 64256, home=bng_path)
bng.open()

In [3]:
## Crear escenario
scenario = Scenario('west_coast_usa', 'ml_beamng')
vehicle = Vehicle('veh', model='pessima', licence='HOLA')
electrics = Electrics()
vehicle.attach_sensor("electrics", electrics)
scenario.add_vehicle(vehicle, pos=(-717, 101, 118), rot_quat=(0, 0, 0.3826834, 0.9238795))
scenario.make(bng)
bng.load_scenario(scenario)
bng.start_scenario()
vehicle.set_shift_mode("arcade")

In [4]:
## Interfaz de grabación
TECLA_GRABAR = 'A'
TECLA_DETENER = 'S'
TECLA_SALIR = 'D'
CARPETA = "automatico"
path = Path(CARPETA)
path.mkdir(exist_ok=True)
file = False

def variables_sensor():
    vehicle.poll_sensors()
    s = "time"
    for data in electrics.data.items():
        if (data[0] == "wheelThermals"):
            continue
        s = s + "," + str(data[0])
    return s

def lecturas_sensor():
    vehicle.poll_sensors()
    s = ""
    for i, data in enumerate(electrics.data.items()):
        if (data[0] == "wheelThermals"):
            continue
        if (i > 0):
            s = s + ","
        s = s + str(data[1])
    return s

while True:
    #manejar teclas
    if (keyboard.is_pressed(TECLA_GRABAR)):
        if (not file):
            filename = CARPETA + "/" + datetime.now().strftime("%m %d %Y - %H %M %S") + ".csv"
            file = open(filename, "w")
            file.write(variables_sensor() + '\n')
            rectime = millis()
            print("Grabando: " + filename)
    elif (keyboard.is_pressed(TECLA_DETENER)):
        if file:
            file.close()
            file = False
            print("Grabado")
    elif (keyboard.is_pressed(TECLA_SALIR)):
        if file:
            file.close()
            file = False
            print("Grabado")
        print("Fuera del loop")
        break
    #leer sensores
    curtime = millis()
    if curtime > (oldtime + step):
        oldtime = curtime
        if file:
            data = lecturas_sensor()
            secs = (curtime - rectime) / 1000
            file.write(str(secs) + "," + data + '\n')

Grabando: automatico/11 20 2022 - 02 33 54.csv
Grabado
Fuera del loop
